In [ ]:
#* for data manipulation 
import pandas as pd 
import numpy as np 

import tiling_disk_plots as tdp
import tiling_disk_utils as tdu

#* sometimes needed for creating bins for histograms 
from astropy.stats import knuth_bin_width

#* for plotting: 
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline
sns.set_theme(style="ticks")
sns.set_context("talk")
%matplotlib inline

### import table map

In [ ]:
table_map = tdu.read_single_measurement('measurements/Tisch', '20220203_table_after_damage_full.txt')
table_map = tdu.add_ring_nr_label(table_map)
table_map = tdu.df_convert_unix_to_datetime(table_map)
# table_map = table_map.query('-158 < z')

# table_map['z'] = tdu.subtract_mean(table_map['z'])
# table_map['z'] = tdu.convert_mm_to_microns(table_map['z'] )
table_map_pt = tdu.point_table(table_map)
table_map_pt = tdu.add_triplet_color_label(table_map_pt)

In [ ]:
table_map_pt

In [ ]:
table_map_pt.z_mean.describe()

In [ ]:
fig, ax = tdp.full_hexagon_plot(table_map_pt, size=5**2, cbar_norm=(-158.070308, -157.999025, -157.869231), cmap='turbo')
ax.set_title('Table measurement')

### Read data

In [ ]:
hardening_tripplets_df = tdu.read_single_measurement('measurements/triplets', '20220209_trip_vels_hardening.txt')
# hardening_tripplets_df = tdu.add_ring_nr_label(hardening_tripplets_df)
hardening_tripplets_df = tdu.df_convert_unix_to_datetime(hardening_tripplets_df)

before_hardening_tripplets_df = tdu.read_single_measurement('measurements/triplets', '20220209_trip_vels_bfg_novac.txt')
# before_hardening_tripplets_df = tdu.add_ring_nr_label(before_hardening_tripplets_df)
before_hardening_tripplets_pt = tdu.point_table(before_hardening_tripplets_df)
before_hardening_tripplets_pt = tdu.add_triplet_color_label(before_hardening_tripplets_pt)

In [ ]:
before_hardening_tripplets_pt.keys()

### Examine pre glue status

In [ ]:
before_hardening_tripplets_pt['z_mean'].describe()

In [ ]:
before_hardening_tripplets_pt.keys()

In [ ]:
fig, ax = tdp.full_hexagon_plot(before_hardening_tripplets_pt, size=5**2, cbar_norm=(-157.056615, -156.981132, -156.879538), cmap='turbo')
ax.set_title('Before Glue: No Vacuum')

# compare before glue to table 

In [ ]:
# table_map_pt.drop('trip_color', axis=1, inplace=True)

In [ ]:
before_hardening_tripplets_pt.drop('trip_color', axis=1, inplace=True)

In [ ]:
ref_sub_table = tdu.combine_mean_measurements(before_hardening_tripplets_pt, table_map_pt)

In [ ]:
ref_sub_table['z_mean'] = tdu.convert_mm_to_microns(ref_sub_table['z_mean'])

In [ ]:
ref_sub_table['z_mean'].describe()

In [ ]:
plt.hist(ref_sub_table['z_mean'])

In [ ]:
tdp.full_hexagon_plot(ref_sub_table, mode=('z_mean'), size=5**2, cbar_norm=(941.692308, 1008.112743, 1104.846154), cmap='turbo', title='Before Glue - Table') #? hier muss auf mm konvertiert werden

# Proceed with before glue as reference

In [ ]:
tdp.plot_data_vs_time(hardening_tripplets_df)

In [ ]:
tdp.ts_hist(hardening_tripplets_df)

### Split Data into runs

In [ ]:
hardening_tripplets_df = hardening_tripplets_df.query('run_nr != 29') # run 29 was incomplete

In [ ]:
measurements_dict = {}
for run in hardening_tripplets_df.run_nr.unique(): 
    data = hardening_tripplets_df.loc[hardening_tripplets_df.run_nr == run, :]
    # print(data)
    measurements_dict[f'run_nr_{run}'] = data

In [ ]:
measurements_dict_pt = {key: tdu.point_table(data) for key, data in measurements_dict.items()}

In [ ]:
for run_nr, data in measurements_dict_pt.items(): 
    data['run_nr'] = run_nr.split('_')[2]

In [ ]:
measurements_dict_pt['run_nr_1']

In [ ]:
# for key, data in measurements_dict_pt.items(): 
#     run_nr = int(key.split('_')[2])
#     fig, ax = tdp.full_hexagon_plot(data, mode=('z_mean'), size=5**2, cbar_norm=(-157.056615, -156.981132, -156.879538), cmap='turbo', title=f'Hour {run_nr -1}')
#     # fig.savefig(f'Vac_timestory/{run_nr}_Measurement.png', dpi='figure', format='png', metadata=None,
#     #     bbox_inches=None, pad_inches=0.1,
#     #     facecolor='auto', edgecolor='auto',
#     #     backend=None
#     #    )
#     # plt.close()

In [ ]:
measurements_dict_pt = {key: tdu.add_triplet_color_label(data) for key, data in measurements_dict_pt.items()}
measurements_dict_pt = {key: tdu.combine_mean_measurements(data, before_hardening_tripplets_pt) for key, data in measurements_dict_pt.items()}

In [ ]:
measurements_dict_pt['run_nr_6']

In [ ]:
for key, data in measurements_dict_pt.items():
    data['z_mean'] = data['z_mean'].apply(lambda z: z*1e3) #* mm -> µm
    data['z_measurement_error'] = data['z_measurement_error'].apply(lambda z: z*1e3) #* mm -> µm
    measurements_dict_pt[key] = data

In [ ]:
measurements_dict_pt['run_nr_1']

In [ ]:
for key, data in measurements_dict_pt.items(): 
    run_nr = int(key.split('_')[2])
    fig, ax = tdp.full_hexagon_plot(data, mode=('z_mean'), size=5**2, cbar_norm=(-50, 0, 50), cmap='turbo', title=f'Hour {run_nr -1}')
    # fig.savefig(f'Vac_timestory/{run_nr}_Measurement.png', dpi='figure', format='png', metadata=None,
    #     bbox_inches=None, pad_inches=0.1,
    #     facecolor='auto', edgecolor='auto',
    #     backend=None
    #    )
    # plt.close()

# hier kommt jetzt der joyplot

erstmal die funktion schreiben um verteilung von einem triplet in einem run zu sehen 

In [ ]:
# for key, data in measurements_dict_pt.items(): 
#     run_nr = int(key.split('_')[2])
#     data = data.loc[data.trip_color=='blue',:]
#     tdp.ts_hist(data, mode='z_mean')


In [ ]:
tdp.plot_triplet_dist_joyplot(measurements_dict_pt, 'blue')

In [ ]:
tdp.plot_triplet_dist_joyplot(measurements_dict_pt, 'brown')

In [ ]:
tdp.plot_triplet_dist_joyplot(measurements_dict_pt, 'yellow')